# Measuring the Galaxy-LyaForest Cross-Correlation from Mock Samples

Here, we calculate the all the Galaxy-Forest cross-correlation in the TreePM256 simulation mocks

Largely adapted from crosscorr_cl2017.ipynb

### Read in Ly-a forest pixels

In [ ]:
import numpy as np
import time as time
import lyafxcorr_kg as xcorr

# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib as mpl
mpl.rc('mathtext',fontset='stixsans')
mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
%matplotlib inline

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

# Define cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.31)

mockdir = '/Users/kheegan/lya/3d_recon/map2017/crosscorr/mocks/'



## Read in bin edges

In [ ]:
PiBin_fil = 'bins23_pi_0-30hMpc.txt'
SigBin_fil = 'bins10_sigma_0-30hMpc.txt'

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)


PiBound = (min(PiEdges), max(PiEdges) )

In [ ]:

for ifile in np.arange(1,81):

    tstart = time.time()
    filesuffix = '{0:03d}'.format(ifile)

    lyafil = mockdir+'pixel_radecz_mock_'+filesuffix+'.bin'
    lyapix = xcorr.lyapix(lyafil,cosmo=cosmo)

    print("Read in {0:d} Ly-a forest pixels from mock {1:03d}".format(lyapix.npix, 
                                                                      ifile))
    
    npix = lyapix.npix

    ### Read in galaxies 
    # We use the catalog created with Create_GalzMocks.IPYNB

    galfil = mockdir+'cat_galmock_nonuniq_'+filesuffix+'.dat'

    gal = ascii.read(galfil, format='ipac')
    #print(gal.columns)

    gal_3d = gal[gal['source'] == '3DHST']
    gal_zD = gal[gal['source'] == 'zDeep']
    gal_mosdef = gal[gal['source'] == 'MOSDEF']
    gal_vuds = gal[gal['source']=='VUDS']
    gal_clamato=gal[gal['source']=='CLAMATO']

    print('Read in %i 3D-HST galaxies, %i zCOSMOS-Deep galaxies, %i MOSDEF galaxies,' 
          '%i VUDS galaxies, %i CLAMATO galaxies' % 
          (len(gal_3d), len(gal_zD), len(gal_mosdef), len(gal_vuds),len(gal_clamato)) )

    # Convert to 3D Sky positions
    Coord_3d     = SkyCoord(ra=gal_3d['ra']  , dec=gal_3d['dec']  ,
                      distance=cosmo.comoving_distance(gal_3d['zspec']))
    Coord_zD     = SkyCoord(ra=gal_zD['ra']  , dec=gal_zD['dec']  ,
                       distance=cosmo.comoving_distance(gal_zD['zspec']))
    Coord_mosdef = SkyCoord(ra=gal_mosdef['ra']  , dec=gal_mosdef['dec']  ,
                       distance=cosmo.comoving_distance(gal_mosdef['zspec']))
    Coord_vuds   = SkyCoord(ra=gal_vuds['ra']  , dec=gal_vuds['dec']  ,
                           distance=cosmo.comoving_distance(gal_vuds['zspec']))
    Coord_clamato=SkyCoord(ra=gal_clamato['ra'], dec=gal_clamato['dec']  ,
                          distance=cosmo.comoving_distance(gal_clamato['zspec']))
    Coord_all = SkyCoord(ra=gal['ra'], dec=gal['dec']  ,
                          distance=cosmo.comoving_distance(gal['zspec']))


    # Compute Cross-Correlation For MOSDEF Galaxies
    XCorr_mosdef, NoXCorr_mosdef = xcorr.xcorr_gal_lya(Coord_mosdef, 
                                                       lyapix, SigEdges, PiEdges, 
                                                       cosmo=cosmo, verbose=0)

    # Compute Cross-Correlation For zCOSMOS-Deep Galaxies
    XCorr_zD, NoXCorr_zD = xcorr.xcorr_gal_lya(Coord_zD, lyapix, 
                                               SigEdges, PiEdges,cosmo=cosmo, verbose=0)


    # Compute Cross-Correlation for CLAMATO Galaxies
    XCorr_clamato, NoXCorr_clamato = xcorr.xcorr_gal_lya(Coord_clamato, lyapix, 
                                                         SigEdges, PiEdges,cosmo=cosmo,
                                                        verbose=0)

    # Compute Cross-Correlation For 3D-HST Galaxies
    XCorr_3d, NoXCorr_3d = xcorr.xcorr_gal_lya(Coord_3d, lyapix, 
                                               SigEdges, PiEdges, cosmo=cosmo, verbose=0)


    # Compute Cross-Correlations for VUDS galaxies
    XCorr_vuds, NoXCorr_vuds = xcorr.xcorr_gal_lya(Coord_vuds, lyapix, 
                                                   SigEdges, PiEdges, cosmo=cosmo, 
                                                   verbose=0)
    
    np.save(mockdir+"crosscorr/xcorrmock_3dhst_"+filesuffix+"_v4.1.npy", XCorr_3d)
    np.save(mockdir+"crosscorr/xcorrmock_zDeep_"+filesuffix+"_v4.1.npy", XCorr_zD)
    np.save(mockdir+"crosscorr/xcorrmock_mosdef_"+filesuffix+"_v4.1.npy", XCorr_mosdef)
    np.save(mockdir+"crosscorr/xcorrmock_vuds_"+filesuffix+"_v4.1.npy", XCorr_vuds)
    np.save(mockdir+"crosscorr/xcorrmock_clamato_"+filesuffix+"_v4.1.npy", XCorr_vuds)

    if ifile ==1:    
        print('Time to generate one mock = {0}s'.format(time.time()-tstart))